In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from tensorflow import keras
from keras import layers
from keras.layers import ReLU
from keras import Sequential

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/data/wildfire_prediction/train',
    labels='inferred',
    label_mode='int',
    class_names=['nowildfire', 'wildfire'],
    color_mode='rgb',
    batch_size=32,
    image_size=(350, 350),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear'
)

Found 19153 files belonging to 2 classes.
Using 3830 files for validation.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/data/wildfire_prediction/valid',
    labels='inferred',
    label_mode='int',
    class_names=['nowildfire', 'wildfire'],
    color_mode='rgb',
    batch_size = 32,
    image_size=(350, 350),
    shuffle=True,
    seed=123,
    interpolation='bilinear'
)

Found 6300 files belonging to 2 classes.


We will take 1024 images from each class from the training data and 512 images for each class from the validation data

In [ ]:
train_ds = train_ds.take(2056)
val_ds = val_ds.take(1012)

**HYPERPARAMETERS**

In [ ]:
epochs = 7
lr = 0.000112
beta_1 = 0.755
beta_2 = 0.992
epsilon = 1e-07
decay = 0.0000256

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Building a Sequential Model**

In [ ]:
def build_model():
  model = Sequential([
    layers.Conv2D(8, 4, padding='same', activation=ReLU()),
    layers.BatchNormalization(),
    layers.AveragePooling2D(),
    layers.Conv2D(16, 4, padding='same', activation=ReLU()),
    layers.BatchNormalization(),
    layers.AveragePooling2D(),
    layers.Conv2D(32, 4, padding='same', activation=ReLU()),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(512, activation=ReLU()),
    layers.Dropout(0.8),
    layers.Dense(124, activation=ReLU()),
    layers.Dropout(0.7),
    layers.Dense(1, activation='sigmoid')
  ])
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, weight_decay = decay)
  model.compile(optimizer=optimizer,
           loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
           metrics=['accuracy'])
  return model

model = build_model()

Train the model

In [ ]:
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs=epochs
)

Epoch 1/7
120/120 [==============================] - 12s 69ms/step - loss: 2.3139 - accuracy: 0.7932 - val_loss: 4.1802 - val_accuracy: 0.6208
Epoch 2/7
120/120 [==============================] - 7s 60ms/step - loss: 1.6078 - accuracy: 0.8311 - val_loss: 1.4791 - val_accuracy: 0.8668
Epoch 3/7
120/120 [==============================] - 7s 60ms/step - loss: 1.1506 - accuracy: 0.8556 - val_loss: 0.9503 - val_accuracy: 0.8913
Epoch 4/7
120/120 [==============================] - 7s 59ms/step - loss: 0.7644 - accuracy: 0.8514 - val_loss: 0.9925 - val_accuracy: 0.8903
Epoch 5/7
120/120 [==============================] - 7s 60ms/step - loss: 0.6073 - accuracy: 0.8640 - val_loss: 0.9888 - val_accuracy: 0.8784
Epoch 6/7
120/120 [==============================] - 7s 60ms/step - loss: 0.5704 - accuracy: 0.8726 - val_loss: 0.8479 - val_accuracy: 0.8829
Epoch 7/7
120/120 [==============================] - 7s 60ms/step - loss: 0.6683 - accuracy: 0.8525 - val_loss: 0.8924 - val_accuracy: 0.9078


Save the model

In [ ]:
model.save('fire_model.h5')